<a href="https://colab.research.google.com/github/KathleenShank/7316/blob/main/Supervised_Classification_GEE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Supervised classification algorithms available in Earth Engine

load starter packages and set up Colab Script

In [ ]:
!pip install geemap
!pip install ipywidgets
!pip install whiteboxgui

import os
import ee
import geemap
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import json
import whiteboxgui

import ipywidgets as widgets
from IPython.display import display

# # Trigger the authentication flow.
ee.Authenticate()

# # Initialize the library.
ee.Initialize(project='shank-kathleen') # insert GEE project name 'shank-kathleen'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.0/74.0 kB 5.7 MB/s eta 0:00:00
Decompressing WhiteboxTools_linux_musl.zip ...
WhiteboxTools package directory: /usr/local/lib/python3.12/dist-packages/whitebox


Create an interactive map

In [4]:
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

Add data to the map - edit ee.ImageCollection to your own image from previous GEE script: LC09_041035_20241010

In [6]:
import ee
ee.Initialize(project='shank-kathleen') # Initialize Earth Engine again just in case

point = ee.Geometry.Point([-122.4439, 37.7538])
# point = ee.Geometry.Point([-87.7719, 41.8799])

image = (
    ee.ImageCollection("LANDSAT/LC09/C02/T1_L2")
    .filterDate("2024-10-10", "2024-10-11")
    .filter(ee.Filter.eq('WRS_PATH', 41))
    .filter(ee.Filter.eq('WRS_ROW', 35))
    .sort("CLOUD_COVER")
    .first()
    .select("SR_B[1-7]") # Surface Reflectance bands are prefixed with SR_
)

vis_params = {"min": 0, "max": 3000, "bands": ["SR_B5", "SR_B4", "SR_B3"]} # Update bands for SR data

# Make sure Map is defined. Assuming geemap is imported and Map is created in a previous cell.
# If Map is also undefined, you would need to run its defining cell too.
Map.centerObject(image, 8)
Map.addLayer(image, vis_params, "Landsat-9")

Check image properties

In [7]:
ee.Date(image.get("system:time_start")).format("YYYY-MM-dd").getInfo()

'2024-10-10'

In [8]:
image.get("CLOUD_COVER").getInfo()

3.13

Make training dataset

In [9]:
# region = Map.user_roi
# region = ee.Geometry.Rectangle([-122.6003, 37.4831, -121.8036, 37.8288])
# region = ee.Geometry.Point([-122.4439, 37.7538]).buffer(10000)

In [10]:
nlcd = ee.Image("USGS/NLCD/NLCD2016").select("landcover").clip(image.geometry())
Map.addLayer(nlcd, {}, "NLCD")
Map

Map(bottom=26026.0, center=[36.039802691874954, -117.89932799589462], controls=(WidgetControl(options=['positi…

In [11]:
# Make the training dataset.
points = nlcd.sample(
    **{
        "region": image.geometry(),
        "scale": 30,
        "numPixels": 5000,
        "seed": 0,
        "geometries": True,  # Set this to False to ignore geometries
    }
)

Map.addLayer(points, {}, "training", False)

In [12]:
print(points.size().getInfo())

5000


In [13]:
print(points.first().getInfo())

{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-118.36125980716353, 36.46595541802843]}, 'id': '0', 'properties': {'landcover': 52}}


Train the classifier

In [18]:
# Use these bands for prediction.
bands = ["SR_B1", "SR_B2", "SR_B3", "SR_B4", "SR_B5", "SR_B6", "SR_B7"]


# This property of the table stores the land cover labels.
label = "landcover"

# Overlay the points on the imagery to get training.
training = image.select(bands).sampleRegions(
    **{"collection": points, "properties": [label], "scale": 30}
)

# Train a CART classifier with default parameters.
trained = ee.Classifier.smileCart().train(training, label, bands)

In [20]:
print(training.first().getInfo())

{'type': 'Feature', 'geometry': None, 'id': '0_0', 'properties': {'SR_B1': 9096, 'SR_B2': 9883, 'SR_B3': 11506, 'SR_B4': 12658, 'SR_B5': 15783, 'SR_B6': 17154, 'SR_B7': 15270, 'landcover': 52}}


Classify the image

In [19]:
# Classify the image with the same bands used for training.
result = image.select(bands).classify(trained)

# # Display the clusters with random colors.
Map.addLayer(result.randomVisualizer(), {}, "classified")
Map

Map(bottom=26026.0, center=[36.039802691874954, -117.89932799589462], controls=(WidgetControl(options=['positi…

Render categorical map

In [21]:
class_values = nlcd.get("landcover_class_values").getInfo()
class_values

[11,
 12,
 21,
 22,
 23,
 24,
 31,
 41,
 42,
 43,
 51,
 52,
 71,
 72,
 73,
 74,
 81,
 82,
 90,
 95]

In [22]:
class_palette = nlcd.get("landcover_class_palette").getInfo()
class_palette

['476ba1',
 'd1defa',
 'decaca',
 'd99482',
 'ee0000',
 'ab0000',
 'b3aea3',
 '68ab63',
 '1c6330',
 'b5ca8f',
 'a68c30',
 'ccba7d',
 'e3e3c2',
 'caca78',
 '99c247',
 '78ae94',
 'dcd93d',
 'ab7028',
 'bad9eb',
 '70a3ba']

In [23]:
landcover = result.set("classification_class_values", class_values)
landcover = landcover.set("classification_class_palette", class_palette)

In [24]:
Map.addLayer(landcover, {}, "Land cover")
Map

Map(bottom=26026.0, center=[36.039802691874954, -117.89932799589462], controls=(WidgetControl(options=['positi…

Visualize the result

In [25]:
print("Change layer opacity:")
cluster_layer = Map.layers[-1]
cluster_layer.interact(opacity=(0, 1, 0.1))

Change layer opacity:


Box(children=(FloatSlider(value=1.0, description='opacity', max=1.0),))

Add a legend to the map

In [26]:
Map.add_legend(builtin_legend="NLCD")
Map

Map(bottom=103558.0, center=[35.646137228802424, -117.36351013183594], controls=(WidgetControl(options=['posit…

Export the result

In [27]:
import os

out_dir = os.path.join(os.path.expanduser("~"), "Downloads")
out_file = os.path.join(out_dir, "landcover.tif")

In [28]:
geemap.ee_export_image(landcover, filename=out_file, scale=900)

Generating URL ...
Please wait ...
An error occurred while downloading.


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

Export the result to Google Drive:

In [ ]:
geemap.ee_export_image_to_drive(
    landcover, description="landcover", folder="export", scale=900
)